# `ApJdataFrames` 
Grankin et al. 2008
---
`Title`: Results of the ROTOR-program     
`Authors`: Grankin et al.  


Data is from this paper:  
http://www.aanda.org/articles/aa/full/2008/09/aa8476-07/aa8476-07.html


In [1]:
import pandas as pd

In [2]:
from astropy.io import ascii, votable, misc

### Download Data

In [3]:
#! mkdir ../data/Grankin08

In [4]:
#! curl http://www.aanda.org/articles/aa/full/2008/09/aa8476-07/table1.tex >> ../data/Grankin08/table1_orig.tex

In [5]:
#! curl http://www.aanda.org/articles/aa/full/2008/09/aa8476-07/table3.tex >> ../data/Grankin08/table3.tex

In [6]:
! du -hs ../data/Grankin08/table1.tex

8.0K	../data/Grankin08/table1.tex


Ew, it's in $\LaTeX$ format!

## Table 1

In [7]:
#! head -n 30 ../data/Grankin08/table1.tex

### Two problems from latex.
1. The values with binaries go onto a new line, which foils data reading.
2. The `\farcs` latex screws up the decimal point.

To deal with these I had to manually delete (horror!) the carriage returns with Sublime Text, then do this:

In [8]:
! cp ../data/Grankin08/table1.tex ../data/Grankin08/table1_mod.tex

In [9]:
! sed -i '' 's/\\farcs/./g' ../data/Grankin08/table1_mod.tex

In [10]:
names_1 = ['Name', 'HBC', 'SpT', 'JD_min_max', 'N_seasons', 'V_range', 'N_obs', 'avgB_V', 'avgV_R', 'mult', 'ref']

In [11]:
tab1 = pd.read_csv('../data/Grankin08/table1_mod.tex', sep='&',
                   skiprows=10, names=names_1, engine='python', skip_footer=8)

In [19]:
tab1.tail()

,Name,HBC,SpT,JD_min_max,N_seasons,V_range,N_obs,avgB_V,avgV_R,mult,ref
43,Wa CrA/2,678,G8 IV,48049--48100,1,10.69--10.53,28,0.85,0.74,,\\
44,Wa Oph/1,630,K2 IV,49145--53228,8,12.19--11.84,190,1.38,1.30,,\\
45,Wa Oph/2,633,K1 IV,51733--53228,5,11.77--11.60,101,1.16,1.11,,\\
46,Wa Oph/3,634,K0 IV,49145--53228,9,10.98--10.73,255,1.19,1.07,,\\
47,Wa Oph/4,652,K4,49147--52059,4,13.72--10.44,104,1.88,1.83,VB (8.7),re93 \\


In [13]:
tab1.to_csv('../data/Grankin08/table1.csv', index=False)

## Table 3

In [14]:
#! tail -n 15 ../data/Grankin08/table3.tex

In [15]:
names = ['Name', 'Epochs', 'delta_V_min', 'delta_V_max', 'HJD0-24000000', 'Period', 'Ref1', 'Ref2']

In [16]:
tab3 = pd.read_csv('../data/Grankin08/table3.tex', sep='&', comment='\\',
                   skiprows=10, names=names, engine='python', skip_footer=5)

In [17]:
tab3.tail()

,Name,Epochs,delta_V_min,delta_V_max,HJD0-24000000,Period,Ref1,Ref2
31,VY Tau,1985--2001,0.11,0.31,44610.7,5.36995,gr91,gr94
32,Wa CrA/1,1990,0.32,0.32,48048.3,2.24000,sh95,sh95
33,Wa CrA/2,1990,0.16,0.16,48048.3,2.79000,co92,sh95
34,Wa Oph/1,1993--2004,0.10,0.30,48986.1,3.79200,za93,this paper
35,Wa Oph/3,1993--2004,0.06,0.23,49144.3,1.52140,sh98,this paper


In [18]:
tab3.to_csv('../data/Grankin08/table3.csv', index=False)

## The end!